# Preparation

In [1]:
import os
import cv2
import shutil
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, Video

In [2]:
OUTPUT_PATH = "/kaggle/working"
DATASET_PATH = "/kaggle/input/snooker-balls/balls"
dataset_path = "/kaggle/working/yolo_dataset"

# Dataset 

### Original Dataset Structure
Dataset structure is given in next format:
```
snooker-balls/balls/
    train/
        {class1}/
            {id_1}.jpg
            {id_2}.jpg
            ...
        {class2}/
            ...
        ...
    test/
    ...
```

### Yolo Dataset Structure
Dataset structure should be transformed to next format:
```
yolo_dataset/
    images/
        train/
            {class1}_{id_1}.jpg
            {class1}_{id_2}.jpg
            ...
        val/
            ...
    labels/
        train/
            {class1}_{id_1}.txt
            {class1}_{id_2}.txt
            ...
        val/
            ...
```

In [3]:
# root directory to save dataset in yolo format
os.makedirs(dataset_path, exist_ok=True)

os.makedirs(f'{dataset_path}/images', exist_ok=True)
os.makedirs(f'{dataset_path}/images/train', exist_ok=True)
os.makedirs(f'{dataset_path}/images/val', exist_ok=True)

os.makedirs(f'{dataset_path}/labels', exist_ok=True)
os.makedirs(f'{dataset_path}/labels/train', exist_ok=True)
os.makedirs(f'{dataset_path}/labels/val', exist_ok=True)

In [4]:
# variables to convert label/id to id/label
label2id = {"black": 0, "blue": 1, "brown": 2, "green": 3, "pink": 4, "red": 5, "white": 6, "yellow": 7}
id2label = {v: k for k, v in label2id.items()}

In [5]:
for layer1 in os.listdir(DATASET_PATH):
    for layer2 in os.listdir(f'{DATASET_PATH}/{layer1}'):
        for layer3 in os.listdir(f'{DATASET_PATH}/{layer1}/{layer2}'):
            if layer3.endswith('.jpg'):
                shutil.copyfile(f'{DATASET_PATH}/{layer1}/{layer2}/{layer3}',f'{dataset_path}/images/{layer1}/{layer2}_{layer3}'.replace('test', 'val' ))
                f = open(f'{dataset_path}/labels/{layer1}/{layer2}_{layer3}'.replace('test', 'val' ).replace('.jpg', '.txt' ), 'w')
                f.write(f'{label2id[layer2]} 0.5 0.5 1 1')
                f.close()

# YOLOv8

In [6]:
!pip install ultralytics==8.0.231
!yolo checks
from ultralytics import YOLO

  Obtaining dependency information for ultralytics==8.0.231 from https://files.pythonhosted.org/packages/98/44/71231f2da4fb4a602d0cef2071adb708199e571ef89ed4a136f59c19d733/ultralytics-8.0.231-py3-none-any.whl.metadata
  Obtaining dependency information for thop>=0.1.1 from https://files.pythonhosted.org/packages/bb/0f/72beeab4ff5221dc47127c80f8834b4bcd0cb36f6ba91c0b1d04a1233403/thop-0.1.1.post2209072238-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 21.7 MB/s eta 0:00:00
Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5563.1/8062.4 GB disk)

OS                  Linux-5.15.133+-x86_64-with-glibc2.31
Environment         Kaggle
Python              3.10.12
Install             pip
RAM                 31.36 GB
CPU                 Intel Xeon 2.20GHz
CUDA                None

matplotlib          ✅ 3.7.4>=3.3.0
numpy               ✅ 1.24.3>=1.22.2
opencv-python       ✅ 4.8.1.78>=4.6.0

In [7]:
names_content = "\n".join([f"  {label_id}: {label}" for label, label_id in label2id.items()])
dataset_content = f"""
path: "{dataset_path}/"
train: "images/train"
val: "images/val"
names:
{names_content}
"""
with open(os.path.join(OUTPUT_PATH, "custom_dataset.yaml"), "w") as f:
    f.write(dataset_content)

In [8]:
# pretrained model: yolov8n、yolov8s、yolov8m、yolov8l、yolov8x
model = YOLO('yolov8n.yaml').load('yolov8n.pt')

# Train the model using the processed dataset
results = model.train(
    data=os.path.join(OUTPUT_PATH,'custom_dataset.yaml'),
    project='snooker_project',
    exist_ok=True,
    epochs=20,
    batch=64,
    imgsz=32,
    optimizer='Adam',
    lr0=0.001,
    lrf=0.0005
)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

100%|██████████| 6.23M/6.23M [00:00<00:00, 137MB/s]

Transferred 355/355 items from pretrained weights
New https://pypi.org/project/ultralytics/8.1.45 available 😃 Update with 'pip install -U ultralytics'


Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/kaggle/working/custom_dataset.yaml, epochs=20, time=None, patience=50, batch=64, imgsz=32, save=True, save_period=-1, cache=False, device=None, workers=8, project=snooker_project, name=train, exist_ok=True, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

100%|██████████| 755k/755k [00:00<00:00, 39.8MB/s]
2024-04-08 23:45:06,686	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-08 23:45:07,875	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


...

      20/20         0G     0.1319      0.131     0.9078         54         32: 100%|██████████| 180/180 [01:22<00:00,  2.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:41<00:00,  1.81s/it]


                   all       2873       2873      0.863      0.941      0.985      0.786

20 epochs completed in 0.597 hours.
Optimizer stripped from snooker_project/train/weights/last.pt, 6.2MB
Optimizer stripped from snooker_project/train/weights/best.pt, 6.2MB

Validating snooker_project/train/weights/best.pt...
Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3007208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:15<00:00,  1.47it/s]


                   all       2873       2873       0.96      0.967      0.985      0.955
                 black       2873        359      0.974      0.932      0.981      0.971
                  blue       2873        323      0.982          1       0.98      0.883
                 brown       2873        331      0.876        0.9      0.972       0.96
                 green       2873        239       0.98          1      0.995      0.993
                  pink       2873         42      0.994          1      0.995      0.989
                   red       2873        322      0.896      0.909      0.968      0.873
                 white       2873       1233      0.997      0.997      0.995      0.972
                yellow       2873         24      0.983          1      0.995      0.995
Speed: 0.0ms preprocess, 1.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to snooker_project/train


lr/pg0,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg2,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
metrics/mAP50(B),▁▄▄█▇▇█▇████████████
metrics/mAP50-95(B),▁▂▃▇▇▇▇▆██▆▆▅▅▆▆▄▅▆█
metrics/precision(B),▃▃▁▄▂▇▇▆▇█▇█▆█▆▇▇▇▇█
metrics/recall(B),▁▃▃▂▂▄▇▆▇█▇█▇██▇██▇█
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▅▄▄▄▃▃▃▃▃▂▂▁▁▁▁▁▁▁▁


In [9]:
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3007208 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 2873 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2873/2873 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:15<00:00,  2.88it/s]


                   all       2873       2873       0.96      0.967      0.985      0.955
                 black       2873        359      0.974      0.932      0.981      0.971
                  blue       2873        323      0.982          1       0.98      0.883
                 brown       2873        331      0.876        0.9      0.972       0.96
                 green       2873        239       0.98          1      0.995      0.993
                  pink       2873         42      0.994          1      0.995      0.989
                   red       2873        322      0.896      0.909      0.968      0.873
                 white       2873       1233      0.997      0.997      0.995      0.972
                yellow       2873         24      0.983          1      0.995      0.995
Speed: 0.0ms preprocess, 1.4ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to snooker_project/train


array([    0.97129,     0.88341,     0.96002,     0.99271,     0.98944,     0.87298,     0.97235,       0.995])

### Model with **ONNX**

In [ ]:
model.export(format='onnx')

In [ ]:
from ultralytics.utils.benchmarks import benchmark
benchmark(model='snooker_project/train/weights/best.pt', imgsz=128)

### Create a Download Link

In [10]:
%cd $OUTPUT_PATH
!zip -r snooker_project.zip snooker_project
from IPython.display import FileLink
FileLink(r'snooker_project.zip')

/kaggle/working
  adding: snooker_project/ (stored 0%)
  adding: snooker_project/train/ (stored 0%)
  adding: snooker_project/train/results.png (deflated 7%)
  adding: snooker_project/train/PR_curve.png (deflated 16%)
  adding: snooker_project/train/train_batch2.jpg (deflated 2%)
  adding: snooker_project/train/labels.jpg (deflated 54%)
  adding: snooker_project/train/val_batch1_labels.jpg (deflated 46%)
  adding: snooker_project/train/args.yaml (deflated 52%)
  adding: snooker_project/train/train_batch1802.jpg (deflated 4%)
  adding: snooker_project/train/train_batch0.jpg (deflated 2%)
  adding: snooker_project/train/weights/ (stored 0%)
  adding: snooker_project/train/weights/last.pt (deflated 44%)
  adding: snooker_project/train/weights/best.pt (deflated 44%)
  adding: snooker_project/train/P_curve.png (deflated 6%)
  adding: snooker_project/train/val_batch0_labels.jpg (deflated 29%)
  adding: snooker_project/train/F1_curve.png (deflated 5%)
  adding: snooker_project/train/train_bat

/kaggle/working/snooker_project.zip

# Predict the Image

In [ ]:
%cd $OUTPUT_PATH
best_model = YOLO("snooker_project/train/weights/best.pt")
best_model.predict(source=image, show=False, save=True, name='image', exist_ok=True)

In [ ]:
%cd $OUTPUT_PATH
plt.imshow(cv2.cvtColor(cv2.imread('runs/detect/image/image1.jpg'), cv2.COLOR_BGR2RGB))
plt.show()

# Video with PyTube

In [ ]:
!pip install pytube
from pytube import YouTube
YouTube('https://youtu.be/hw02UKK4Kb0').streams.filter().get_highest_resolution().download(output_path=OUTPUT_PATH, filename='youtube.mp4')

In [ ]:
%cd $OUTPUT_PATH
!ffmpeg -i youtube.mp4 -vcodec copy -acodec copy -ss 00:01:05 -to 00:01:15 video.mp4 -y

# Predict the Video

In [ ]:
%cd $OUTPUT_PATH
best_model = YOLO("snooker_project/train/weights/best.pt")
best_model.predict(source=video, show=False, save=True, name='predict', exist_ok=True)

In [ ]:
%cd $OUTPUT_PATH
!ffmpeg -y -loglevel panic -i runs/detect/predict/video.avi predict_video.mp4
Video("predict_video.mp4", width=840)

# Predict with **SAHI**

In [24]:
%cd $OUTPUT_PATH
# Clone the ultralytics repository
!git clone https://github.com/ultralytics/ultralytics.git 

/kaggle/working
fatal: destination path 'ultralytics' already exists and is not an empty directory.


In [18]:
!pip install -U sahi

  Obtaining dependency information for sahi from https://files.pythonhosted.org/packages/0b/da/23f351eb3360e58762f0d1ab2dc8521610cefb9e30e246eb715cbe337a38/sahi-0.11.15-py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python<=4.8 from https://files.pythonhosted.org/packages/29/35/a791b550cdeb4efd8b66e921748f2aff938868a29794489d93575d604a02/opencv_python-4.7.0.72-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pybboxes==0.1.6 from https://files.pythonhosted.org/packages/3c/3f/46f6613b41a3c2b4f7af3b526035771ca5bb12d6fdf3b23145899f785e36/pybboxes-0.1.6-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for terminaltables from https://files.pythonhosted.org/packages/c4/fb/ea621e0a19733e01fe4005d46087d383693c0f4a8f824b47d8d4122c87e0/terminaltables-3.1.10-py2.py3-none-any.whl.metadata

In [29]:
%cd $OUTPUT_PATH
%cd ultralytics/examples/YOLOv8-SAHI-Inference-Video

!python yolov8_sahi.py --source "/kaggle/input/snooker-private-test/video.mp4" --save-img --weights "/kaggle/working/snooker_project/train/weights/best.pt"

/kaggle/working
/kaggle/working/ultralytics/examples/YOLOv8-SAHI-Inference-Video
Traceback (most recent call last):
  File "/kaggle/working/ultralytics/examples/YOLOv8-SAHI-Inference-Video/yolov8_sahi.py", line 7, in <module>
    from sahi import AutoDetectionModel
  File "/opt/conda/lib/python3.10/site-packages/sahi/__init__.py", line 3, in <module>
    from sahi.annotation import BoundingBox, Category, Mask
  File "/opt/conda/lib/python3.10/site-packages/sahi/annotation.py", line 9, in <module>
    from sahi.utils.coco import CocoAnnotation, CocoPrediction
  File "/opt/conda/lib/python3.10/site-packages/sahi/utils/coco.py", line 17, in <module>
    from shapely import MultiPolygon
ImportError: cannot import name 'MultiPolygon' from 'shapely' (/opt/conda/lib/python3.10/site-packages/shapely/__init__.py)


In [ ]:
# from sahi.utils.yolov8 import download_yolov8s_model
# from sahi import AutoDetectionModel
# from sahi.utils.cv import read_image
# from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
# from pathlib import Path

In [ ]:
model_type = "yolov8"
model_path = f"{OUTPUT_PATH}/snooker_project/train/weights/best.pt"
model_device = "cpu" # or 'cuda:0'
model_confidence_threshold = 0.4

slice_height = 32
slice_width = 32
overlap_height_ratio = 0.2
overlap_width_ratio = 0.2

image = '/kaggle/input/snooker-private-test/image1.jpg'
video = '/kaggle/input/snooker-private-test/video.mp4'

In [ ]:
%cd $OUTPUT_PATH
detection_model = AutoDetectionModel.from_pretrained(
    model_type=model_type,
    model_path=model_path,
    device=model_device,
    confidence_threshold=model_confidence_threshold,
)

In [ ]:
result = get_sliced_prediction(
    image,
    detection_model,
    slice_height=slice_height,
    slice_width=slice_width,
    overlap_height_ratio=overlap_height_ratio,
    overlap_width_ratio=overlap_width_ratio
)

In [ ]:
# best_model.predict(source=video, show=False, save=True, name='predict', exist_ok=True)
predict(
    source=video, # "path/to/dir"
    model_type=model_type,
    model_path=model_path,
    model_device=model_device,
    model_confidence_threshold=model_confidence_threshold,
    slice_height=slice_height,
    slice_width=slice_width,
    overlap_height_ratio=overlap_height_ratio,
    overlap_width_ratio=overlap_width_ratio,
)

In [ ]:
result.export_visuals(export_dir=OUTPUT_PATH)
Image(f"{OUTPUT_PATH}/prediction_visual.png")

# Track the Video with **ByteTrack**

In [ ]:
%cd $OUTPUT_PATH
best_model = YOLO('snooker_project/train/weights/best.pt')
best_model.track(source=video, tracker="bytetrack.yaml", save=True, name='track', exist_ok=True)

In [ ]:
%cd $OUTPUT_PATH
!ffmpeg -y -loglevel panic -i runs/detect/track/video.avi track_video.mp4
Video("track_video.mp4", width=840)